In [11]:
# Función global para limpiar índices y columnas residuales antes de unir DataFrames
def limpiar_indices(df):
    for col in ['index', 'level_0']:
        if col in df.columns:
            df = df.drop(columns=[col])
    return df.reset_index(drop=True)

# Lectura Claro

In [13]:
from selenium import webdriver
from selenium.webdriver.edge.options import Options
from selenium.webdriver.common.by import By
import pandas as pd
import time

options = Options()
options.add_argument("--remote-allow-origins=*")

driver = webdriver.Edge(options=options)

datos = []
# Recorremos varias páginas para obtener todos los planes de Claro (solo texto completo, sin filtrar)
for pagina in range(1, 6):  # Cambia 6 por más si hay más páginas
    url = f"https://www.tiendaclaro.pe/personas/hogar/internet?page={pagina}"
    driver.get(url)
    print(f"🌀 Cargando página {pagina}...")
    time.sleep(8)  # Esperar carga inicial

    # Scroll profundo para que cargue más productos
    for _ in range(8):
        driver.execute_script("window.scrollBy(0, 1000);")
        time.sleep(1.5)

    # Extraer todos los contenedores de planes (solo texto completo)
    secciones = driver.find_elements(By.CSS_SELECTOR, "section.vtex-product-summary-2-x-container")

    for sec in secciones:
        try:
            texto = sec.text.strip()
        except:
            texto = "No disponible"
        if texto:
            datos.append({"Contenido completo del plan": texto})

driver.quit()

# Crear DataFrame solo con el texto completo de cada plan, sin filtrar ni procesar
df_claro = pd.DataFrame(datos).drop_duplicates()
df_claro = df_claro[df_claro["Contenido completo del plan"].str.strip() != ""]  # Eliminar vacíos
df_claro = limpiar_indices(df_claro)
print(df_claro)


ModuleNotFoundError: No module named 'pandas'

# Filtrado CLARO

In [39]:
#Hacer un filtrado de los planes de Claro
import re
import pandas as pd

def extraer_datos_claro(texto):     
    # Definir patrones de búsqueda
    patrones = {
        "Nombre del plan": r"(?i)(?:Plan|Paquete|Oferta|Combo)\s*:\s*([^\n]+)",
        "Precio": r"(?i)(?:Precio|Valor)\s*:\s*([^\n]+)",
        "Velocidad de bajada": r"(?i)(?:Velocidad de bajada|Descarga)\s*:\s*([^\n]+)",
        "Velocidad de subida": r"(?i)(?:Velocidad de subida|Carga)\s*:\s*([^\n]+)",
        "Datos móviles": r"(?i)(?:Datos móviles|Internet móvil)\s*:\s*([^\n]+)",
        "Llamadas ilimitadas": r"(?i)(?:Llamadas ilimitadas|Voz ilimitada)\s*:\s*([^\n]+)",
        "SMS ilimitados": r"(?i)(?:SMS ilimitados|Mensajes ilimitados)\s*:\s*([^\n]+)",
        "Duración del contrato": r"(?i)(?:Duración del contrato|Compromiso)\s*:\s*([^\n]+)"
    }
    
    # Extraer datos usando los patrones
    datos = {}
    for clave, patron in patrones.items():
        resultado = re.search(patron, texto)
        if resultado:
            datos[clave] = resultado.group(1).strip()
        else:
            datos[clave] = None  # Si no se encuentra, asignar None
    
    return datos

# Filtrado robusto para planes de Claro: Tipo de plan, Precio regular, Velocidad regular, Tecnología, Extras
def limpiar_numero(texto):
    # Elimina espacios, separadores de miles y deja solo el punto decimal
    texto = texto.replace(' ', '').replace('\xa0', '')
    # Si hay tanto punto como coma, asume que la coma es decimal (formato europeo)
    if ',' in texto and '.' in texto:
        if texto.rfind(',') > texto.rfind('.'):
            texto = texto.replace('.', '').replace(',', '.')
        else:
            texto = texto.replace(',', '')
    else:
        texto = texto.replace(',', '.')
    try:
        return float(texto)
    except Exception:
        return None

def filtrar_claro(texto):
    texto = texto.replace('\r', '').replace('\t', '').strip()
    lineas = [l.strip() for l in texto.split('\n') if l.strip()]

    # Tipo de plan: buscar "Hogar X Play" o similar en la primera línea
    tipo_plan = "No indica"
    if lineas:
        match_tipo = re.search(r"hogar\s*\d+\s*play", lineas[0], re.IGNORECASE)
        if match_tipo:
            tipo_plan = match_tipo.group(0).title()
        else:
            tipo_plan = lineas[0]

    # Precio regular: buscar el mayor S/ en el texto, limpiando correctamente los valores
    precios = re.findall(r"s/?\s*([\d.,\s]+)", texto, re.IGNORECASE)
    precios_float = [limpiar_numero(p) for p in precios if limpiar_numero(p) is not None]
    precio_regular = "No indica"
    if precios_float:
        precio_regular = f"S/ {max(precios_float):.2f}"

    # Velocidad regular: buscar la mayor velocidad en Mbps (soporta formatos con espacios y comas)
    velocidades = re.findall(r"(\d{2,4}[.,]?\d*)\s*mbps", texto, re.IGNORECASE)
    velocidades_float = [limpiar_numero(v) for v in velocidades if limpiar_numero(v) is not None]
    velocidad_regular = "No indica"
    if velocidades_float:
        velocidad_regular = f"{max(velocidades_float):.0f} Mbps"

    # Tecnología: buscar "fibra" o "cobre" o "hfc"
    tecnologia = "No indica"
    if re.search(r"fibra", texto, re.IGNORECASE):
        tecnologia = "Fibra óptica"
    elif re.search(r"hfc", texto, re.IGNORECASE):
        tecnologia = "HFC"
    elif re.search(r"cobre|adsl", texto, re.IGNORECASE):
        tecnologia = "ADSL/Cobre"

    # Extras: buscar bonos, meses gratis, promociones, etc.
    extras = []
    for l in lineas:
        l_low = l.lower()
        if "+ bono" in l_low or "bono" in l_low:
            extras.append(l)
        if "mes gratis" in l_low or "x 6 meses" in l_low or "x 3 meses" in l_low or "x 1 mes" in l_low:
            extras.append(l)
        if "descubre" in l_low or "nuevas velocidades" in l_low:
            extras.append(l)
        if "para toda la vida" in l_low:
            extras.append(l)
        if "play" in l_low and l != lineas[0]:
            extras.append(l)
        if "para toda la vida" in l_low:
            extras.append(l)
        if "promoción" in l_low or "promo" in l_low:
            extras.append(l)
        if "canal" in l_low:
            extras.append(l)
    extras = list(dict.fromkeys(extras))  # Eliminar duplicados
    extras_str = " / ".join(extras) if extras else "Ninguno"

    return {
        "Tipo de plan": tipo_plan,
        "Precio regular": precio_regular,
        "Velocidad regular": velocidad_regular,
        "Tecnología": tecnologia,
        "Extras": extras_str
    }

# Aplicar a tu DataFrame de Claro
df_filtrado_claro = df_claro["Contenido completo del plan"].apply(filtrar_claro).apply(pd.Series)
df_filtrado_claro = limpiar_indices(df_filtrado_claro)
print(df_filtrado_claro.head())


   Tipo de plan Precio regular Velocidad regular Tecnología  \
0  Hogar 3 Play      S/ 800.00         1000 Mbps  No indica   
1  Hogar 1 Play      S/ 600.00         1000 Mbps  No indica   
2  Hogar 2 Play      S/ 600.00         1000 Mbps  No indica   
3  Hogar 2 Play      S/ 600.00         1000 Mbps  No indica   
4  Hogar 1 Play      S/ 800.00         1000 Mbps  No indica   

                                              Extras  
0  + BONO 1000 MBPS / x 6 meses / para toda la vi...  
1  + BONO 1000 MBPS / x 6 meses / para toda la vi...  
2  + BONO 1000 MBPS / x 6 meses / para toda la vi...  
3  + BONO 1000 MBPS / x 6 meses / para toda la vi...  
4  + BONO 1000 MBPS / x 6 meses / para toda la vi...  


# Lectura MOVISTAR

In [ ]:
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from selenium.webdriver.edge.options import Options
from selenium.webdriver.common.by import By
import pandas as pd
import time

# Configuración de Selenium para Edge
options = Options()
# Descomenta la siguiente línea si quieres ejecutar en modo headless (sin ventana)
# options.add_argument("--headless=new")
options.add_argument("--disable-gpu")

service = Service(executable_path="C:/WebDrivers/msedgedriver.exe")

urls = {
    "Trio": "https://movistaronline.pe/hogar/plan-trio",
    "Duo": "https://movistaronline.pe/hogar/plan-duo",
    "Solo": "https://movistaronline.pe/hogar/internet-solo"
}

datos = []

try:
    driver = webdriver.Edge(service=service, options=options)
    for tipo, url in urls.items():
        print(f"🔍 Procesando {tipo}...")
        driver.get(url)
        # Espera explícita para asegurar carga de la página
        time.sleep(8)

        # Scroll para cargar dinámicamente los planes
        for _ in range(6):
            driver.execute_script("window.scrollBy(0, 800);")
            time.sleep(1.2)

        # Selector específico para cada plan
        bloques = driver.find_elements(By.CSS_SELECTOR, "div.cajaOferta")

        for bloque in bloques:
            texto = bloque.text.strip()
            if len(texto) < 50:
                continue  # Ignorar bloques vacíos o irrelevantes
            datos.append({
                "Tipo de plan": tipo,
                "Contenido del plan": texto
            })
except Exception as e:
    print(f"❌ Error durante el scraping: {e}")
finally:
    try:
        driver.quit()
    except Exception:
        pass

# Guardar resultado en DataFrame
df = pd.DataFrame(datos).drop_duplicates()
print(df)

df_movistar = df.copy()


🔍 Procesando Trio...
🔍 Procesando Duo...
🔍 Procesando Duo...
🔍 Procesando Solo...
🔍 Procesando Solo...
   Tipo de plan                                 Contenido del plan
0          Trio  PLAN RECOMENDADO\nInternet Fibra Óptica\n400 M...
1          Trio  Internet Fibra Óptica\n600 Mbps\nVelocidad sim...
2          Trio  Internet Fibra Óptica\n1000 Mbps\nVelocidad si...
3          Trio  Internet Fibra Óptica\n200 Mbps\nVelocidad sim...
4           Duo  PLAN RECOMENDADO\nInternet + TV\nFibra Óptica\...
5           Duo  Internet + TV\nFibra Óptica\n600 Mbps\nVelocid...
6           Duo  Internet + TV\nFibra Óptica\n1000 Mbps\nVeloci...
7           Duo  Internet + TV\nFibra Óptica\n200 Mbps\nVelocid...
8          Solo  PLAN RECOMENDADO\nInternet Fibra Óptica\n400 M...
9          Solo  Internet Fibra Óptica\n600 Mbps\nVelocidad sim...
10         Solo  Internet Fibra Óptica\n1000 Mbps\nVelocidad si...
11         Solo  Internet Fibra Óptica\n200 Mbps\nVelocidad sim...
   Tipo de plan           

## Filtrado MOVISTAR 

In [ ]:
import re
import pandas as pd

def extraer_datos(texto):
    texto = texto.lower()

    # Tipo de tecnología
    tecnologia = "No indica"
    if "fibra" in texto:
        tecnologia = "Fibra óptica"
    elif "adsl" in texto or "cobre" in texto:
        tecnologia = "ADSL/Cobre"
    elif "hfc" in texto:
        tecnologia = "HFC"

    # Precio regular
    precio_regular = "No indica"
    match_regular = re.search(r"precio regular\s*s/[\s]*([\d.,]+)", texto)
    if match_regular:
        precio_regular = f"S/ {match_regular.group(1)}"
    else:
        match_alt = re.search(r"s/[\s]*([\d.,]+)\s*al\s*mes", texto)
        if match_alt:
            precio_regular = f"S/ {match_alt.group(1)}"

    # Velocidad regular
    vel_regular = "No indica"
    match_vel = re.search(r"(\d+)\s*mbps", texto)
    if match_vel:
        vel_regular = match_vel.group(1) + " Mbps"

    # Extras
    extras = []
    if "movistar tv app lite" in texto:
        extras.append("Movistar TV App Lite")
    elif "movistar tv app" in texto:
        extras.append("Movistar TV App")
    if "teléfono fijo" in texto or "equipo fijo" in texto:
        extras.append("Teléfono fijo")
    if "instalación gratis" in texto:
        extras.append("Instalación gratis")
    if "sin costo adicional" in texto:
        extras.append("Sin costo adicional")
    if "x12 meses" in texto:
        extras.append("Beneficio x12 meses")

    return {
        "Tipo de plan": None,  # Se llenará luego
        "Precio regular": precio_regular,
        "Velocidad regular": vel_regular,
        "Tecnología": tecnologia,
        "Extras": " / ".join(extras) if extras else "Ninguno"
    }

# Aplicar a tu DataFrame
df_filtrado_movistar = pd.concat([
    df_movistar[["Tipo de plan"]],
    df_movistar["Contenido del plan"].apply(extraer_datos).apply(pd.Series)
], axis=1)

# Rellenar el tipo de plan si está vacío
df_filtrado_movistar["Tipo de plan"] = df_filtrado_movistar["Tipo de plan"]\
    .fillna(df_movistar["Tipo de plan"])

# Limpiar índices antes de unir
df_filtrado_movistar = limpiar_indices(df_filtrado_movistar)

# Mostrar y guardar
print(df_filtrado_movistar)

   Tipo de plan Tipo de plan Precio regular Velocidad regular    Tecnología  \
0          Trio         None      S/ 179.90          400 Mbps  Fibra óptica   
1          Trio         None      S/ 199.90          600 Mbps  Fibra óptica   
2          Trio         None      S/ 239.90         1000 Mbps  Fibra óptica   
3          Trio         None      S/ 159.90          200 Mbps  Fibra óptica   
4           Duo         None      S/ 169.90          400 Mbps  Fibra óptica   
5           Duo         None      S/ 189.90          600 Mbps  Fibra óptica   
6           Duo         None      S/ 229.90         1000 Mbps  Fibra óptica   
7           Duo         None      S/ 149.90          200 Mbps  Fibra óptica   
8          Solo         None       S/ 89.90          400 Mbps  Fibra óptica   
9          Solo         None      S/ 109.90          600 Mbps  Fibra óptica   
10         Solo         None      S/ 149.90         1000 Mbps  Fibra óptica   
11         Solo         None       S/ 69.90         

# Lectura WoW

In [ ]:
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from selenium.webdriver.edge.options import Options
from selenium.webdriver.common.by import By
import pandas as pd
import time

# Configuración de Selenium para Edge
options = Options()
# options.add_argument("--headless=new")  # Descomenta si deseas ocultar el navegador
options.add_argument("--disable-gpu")

# Ruta al ejecutable msedgedriver (ajusta si es diferente en tu PC)
service = Service(executable_path="C:/WebDrivers/msedgedriver.exe")

datos = []
try:
    driver = webdriver.Edge(service=service, options=options)
    # Ir a la página
    url = "https://wowperu.pe/planes/"
    driver.get(url)
    time.sleep(8)  # Esperar carga inicial

    # Hacer scroll para asegurar carga de todos los elementos
    for _ in range(3):
        driver.execute_script("window.scrollBy(0, 800);")
        time.sleep(1.2)

    # Selector robusto para los planes
    contenedores = driver.find_elements(By.XPATH, "//div[contains(@class, 'p-2') and contains(@class, 'lg:p-4')]")

    for contenedor in contenedores:
        try:
            texto = contenedor.text.strip()
            if len(texto) > 50:
                datos.append({"Contenido del plan": texto})
        except Exception as e:
            print("⚠️ Error al leer un contenedor:", e)
except Exception as e:
    print(f"❌ Error durante el scraping de WOW: {e}")
finally:
    try:
        driver.quit()
    except Exception:
        pass

# Crear y guardar DataFrame
df = pd.DataFrame(datos).drop_duplicates()
df_wow = df.copy()
print(df_wow)

                                  Contenido del plan
0  Velocidad Contratada 100Mbps\n200Mbps\nx 6 mes...
1  Velocidad Contratada 200Mbps\n400Mbps\nx 6 mes...
2  PROMO SOLO DIGITAL\n1000Mbps\n+30 canales (8)\...
3  (4)\n+32\ncanales(8)\nPrecio promocional desde...
4  Básico\n(5)\n+80\ncanales(8)\nPrecio promocion...
5  Full\n(6)\nIncluye\n+100\ncanales(8)\nPrecio p...


## Filtrado Wow

In [ ]:
# Filtrado mejorado para planes WOW: columnas requeridas
import re
import pandas as pd

def limpiar_numero(texto):
    texto = texto.replace(' ', '').replace('\xa0', '')
    if ',' in texto and '.' in texto:
        if texto.rfind(',') > texto.rfind('.'):
            texto = texto.replace('.', '').replace(',', '.')
        else:
            texto = texto.replace(',', '')
    else:
        texto = texto.replace(',', '.')
    try:
        return float(texto)
    except Exception:
        return None

def extraer_datos_wow(texto):
    texto = texto.replace('\r', '').replace('\t', '').strip().lower()
    lineas = [l.strip() for l in texto.split('\n') if l.strip()]

    # Tipo de plan: solo puede ser Básico, Duo, Trio o No indica, nunca la tecnología
    tipo_plan = "No indica"
    for l in lineas:
        if "básico" in l and not any(x in l for x in ["fibra", "hfc", "adsl", "cobre"]):
            tipo_plan = "Básico"
            break
        elif "duo" in l and not any(x in l for x in ["fibra", "hfc", "adsl", "cobre"]):
            tipo_plan = "Duo"
            break
        elif "trio" in l and not any(x in l for x in ["fibra", "hfc", "adsl", "cobre"]):
            tipo_plan = "Trio"
            break

    # Asegurarse que el tipo de plan sea uno de los valores aceptados
    if tipo_plan not in ["básico", "duo", "trio", "no indica"]:
        tipo_plan = "No indica"

    # Precio regular: buscar el mayor S/ o "precio promocional desde"
    precios = re.findall(r"s?/\s*([\d.,\s]+)", texto)
    precios_float = [limpiar_numero(p) for p in precios if limpiar_numero(p) is not None]
    precio_promocional = re.findall(r"precio promocional desde\s*s?/\s*([\d.,\s]+)", texto)
    if precio_promocional:
        precios_float += [limpiar_numero(p) for p in precio_promocional if limpiar_numero(p) is not None]
    precio_regular = "No indica"
    if precios_float:
        precio_regular = f"S/ {max(precios_float):.2f}"

    # Velocidad regular: buscar la mayor velocidad en Mbps (evitar confusión con canales)
    velocidades = []
    for l in lineas:
        # Solo considerar líneas que contengan 'mbps' y no 'canal'
        if 'mbps' in l and 'canal' not in l:
            vs = re.findall(r"(\d{2,4}[.,]?\d*)\s*mbps", l)
            velocidades.extend([limpiar_numero(v) for v in vs if limpiar_numero(v) is not None])
    velocidad_regular = "No indica"
    if velocidades:
        velocidad_regular = f"{max(velocidades):.0f} Mbps"

    # Tecnología: siempre Fibra óptica para WOW
    tecnologia = "Fibra óptica"

    # Extras: canales, promociones, meses gratis, etc.
    extras = []
    canales = re.findall(r"\+\s*(\d+)\s*canales", texto)
    if canales:
        extras.append(f"Incluye {canales[0]} canales")
    if "mes gratis" in texto:
        extras.append("Mes gratis")
    if "x 6 meses" in texto:
        extras.append("Promoción x 6 meses")
    if "x 3 meses" in texto:
        extras.append("Promoción x 3 meses")
    if "x 1 mes" in texto or "x1 mes" in texto:
        extras.append("Promoción x 1 mes")
    if "pago puntual" in texto:
        extras.append("Precio con pago puntual")
    if "dto" in texto or "descuento" in texto:
        extras.append("Descuento especial")
    if "precio promocional" in texto:
        extras.append("Precio promocional")
    extras = list(dict.fromkeys(extras))
    extras_str = " / ".join(extras) if extras else "Ninguno"

    return {
        "Tipo de plan": tipo_plan,
        "Precio regular": precio_regular,
        "Velocidad regular": velocidad_regular,
        "Tecnología": tecnologia,
        "Extras": extras_str
    }

# Aplicar a tu DataFrame WOW
df_filtrado_wow = df_wow["Contenido del plan"].apply(extraer_datos_wow).apply(pd.Series)
df_filtrado_wow = limpiar_indices(df_filtrado_wow)
print(df_filtrado_wow.head())

  Tipo de plan Precio regular Velocidad regular    Tecnología  \
0    No indica       S/ 69.90          200 Mbps  Fibra óptica   
1    No indica       S/ 74.90          400 Mbps  Fibra óptica   
2    No indica       S/ 79.90         1000 Mbps  Fibra óptica   
3    No indica       S/ 93.90         No indica  Fibra óptica   
4    No indica      S/ 124.90         No indica  Fibra óptica   

                                              Extras  
0  Incluye 30 canales / Promoción x 6 meses / Pre...  
1  Incluye 30 canales / Promoción x 6 meses / Pre...  
2  Incluye 30 canales / Promoción x 6 meses / Pro...  
3  Incluye 32 canales / Promoción x 6 meses / Pre...  
4  Incluye 80 canales / Promoción x 6 meses / Pre...  


# Lectura WIN

In [ ]:
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from selenium.webdriver.edge.options import Options
from selenium.webdriver.common.by import By
import pandas as pd
# Configuración de Selenium para Edge
options = Options()
# options.add_argument("--headless=new")  # Descomenta si deseas sin ventana
options.add_argument("--disable-gpu")

# Ruta al WebDriver de Edge (ajusta si tu ruta es distinta)
service = Service(executable_path="C:/WebDrivers/msedgedriver.exe")

datos = []
try:
    driver = webdriver.Edge(service=service, options=options)
    # Ir a la página de WIN
    url = "https://internet-fibra.pe/p/win-productos/win-internet-fibra.html"
    driver.get(url)
    time.sleep(8)  # Esperar que cargue todo

    # Scroll por si el contenido es dinámico (seguridad extra)
    for _ in range(3):
        driver.execute_script("window.scrollBy(0, 800);")
        time.sleep(1.2)

    # Selector principal de los planes
    planes = driver.find_elements(By.CSS_SELECTOR, "div.card__plan")

    for plan in planes:
        try:
            texto = plan.text.strip()
            if texto:
                datos.append({"Contenido del plan": texto})
        except Exception as e:
            print("⚠️ Error en un contenedor:", e)
except Exception as e:
    print(f"❌ Error durante el scraping de WIN: {e}")
finally:
    try:
        driver.quit()
    except Exception:
        pass

# Crear DataFrame y guardar
df_win = pd.DataFrame(datos).drop_duplicates()
print(df_win.head())

                                  Contenido del plan
0  100% Fibra Óptica Internet\n200 Mbps\n400 Mbps...
1  Plan Recomendado\n100% Fibra Óptica Internet\n...
2  100% Fibra Óptica Internet\n400 Mbps\n800 Mbps...
3  100% Fibra Óptica Internet\n600 Mbps\nS/109 x ...
4  100% Fibra Óptica Internet\n1000 Mbps\nS/129 x...


## FIltrado WIN


In [61]:
# Codigo mejorado para filtrar WIN
import re
import pandas as pd

def extraer_datos_win(texto):
    texto = texto.lower()
    lineas = [l.strip() for l in texto.split('\n') if l.strip()]

    # Tipo de plan: solo puede ser Básico, Full, Promoción, Recomendado o No indica, nunca la tecnología
    tipo_plan = "No indica"
    for l in lineas:
        if "plan recomendado" in l and not any(x in l for x in ["fibra", "hfc", "adsl", "cobre"]):
            tipo_plan = "Recomendado"
            break
        elif "básico" in l and not any(x in l for x in ["fibra", "hfc", "adsl", "cobre"]):
            tipo_plan = "Básico"
            break
        elif "full" in l and not any(x in l for x in ["fibra", "hfc", "adsl", "cobre"]):
            tipo_plan = "Full"
            break
        elif "promo" in l and not any(x in l for x in ["fibra", "hfc", "adsl", "cobre"]):
            tipo_plan = "Promoción"
            break
    # Si el tipo de plan es una tecnología, forzar 'No indica'
    if tipo_plan.lower() in ["fibra óptica", "fibra optica", "hfc", "adsl", "cobre"]:
        tipo_plan = "No indica"

    # Precio regular: buscar explícito o el mayor valor S/ en el texto
    precio_regular = "No indica"
    match_precio = re.search(r"precio regular\s*s?/\s*([\d.,]+)", texto)
    if match_precio:
        precio_regular = f"S/ {match_precio.group(1)}"
    else:
        precios = re.findall(r"s?/\s*([\d.,]+)", texto)
        precios_float = [float(p.replace(',', '.')) for p in precios if p]
        if precios_float:
            precio_regular = f"S/ {max(precios_float):.2f}" if len(precios_float) > 0 else "No indica"

    # Velocidad regular: buscar la menor si hay promo, la mayor si no
    velocidades = re.findall(r"(\d{2,4})\s*mbps", texto)
    velocidades_int = [int(v) for v in velocidades]
    velocidad_regular = "No indica"
    if velocidades_int:
        if "x 6 meses" in texto or "x 3 meses" in texto:
            velocidad_regular = f"{min(velocidades_int)} Mbps"
        else:
            velocidad_regular = f"{max(velocidades_int)} Mbps"

    # Tecnología
    tecnologia = "No indica"
    if "fibra" in texto:
        tecnologia = "Fibra óptica"
    elif "adsl" in texto or "cobre" in texto:
        tecnologia = "ADSL/Cobre"
    elif "hfc" in texto:
        tecnologia = "HFC"

    # Extras
    extras = []
    if "x 6 meses" in texto:
        extras.append("Promoción x 6 meses")
    if "x 3 meses" in texto:
        extras.append("Promoción x 3 meses")
    if "mes gratis" in texto:
        extras.append("Mes gratis")
    if "instalación gratis" in texto:
        extras.append("Instalación gratis")
    if "consultar cobertura" in texto:
        extras.append("Consultar cobertura")
    if "plan recomendado" in texto:
        extras.append("Plan recomendado")
    if "precio regular" not in texto and "s/" in texto and "x mes" in texto:
        extras.append("Precio promocional temporal")
    extras = list(dict.fromkeys(extras))
    extras_str = " / ".join(extras) if extras else "Ninguno"

    return {
        "Tipo de plan": tipo_plan,
        "Precio regular": precio_regular,
        "Velocidad regular": velocidad_regular,
        "Tecnología": tecnologia,
        "Extras": extras_str
    }

#Aplicar a tu DataFrame WIN
df_filtrado_win = df_win["Contenido del plan"].apply(extraer_datos_win).apply(pd.Series)
df_filtrado_win = limpiar_indices(df_filtrado_win)
# Mostrar y guardar el DataFrame filtrado
print(df_filtrado_win)

  Tipo de plan Precio regular Velocidad regular    Tecnología  \
0    No indica       S/ 99.00          200 Mbps  Fibra óptica   
1  Recomendado         S/ 119          300 Mbps  Fibra óptica   
2    No indica         S/ 129          400 Mbps  Fibra óptica   
3    No indica         S/ 139          600 Mbps  Fibra óptica   
4    No indica         S/ 159         1000 Mbps  Fibra óptica   

                                              Extras  
0  Promoción x 6 meses / Consultar cobertura / Pr...  
1  Promoción x 6 meses / Promoción x 3 meses / Co...  
2  Promoción x 6 meses / Promoción x 3 meses / Co...  
3          Promoción x 3 meses / Consultar cobertura  
4          Promoción x 3 meses / Consultar cobertura  


# Lectura Entel

In [47]:
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from selenium.webdriver.edge.options import Options
from selenium.webdriver.common.by import By
import pandas as pd
import time

# Configuración de Edge
options = Options()
# options.add_argument("--headless=new")  # Opcional: sin ventana
options.add_argument("--disable-gpu")

# Ruta al msedgedriver (ajústala si es distinta)
service = Service(executable_path="C:/WebDrivers/msedgedriver.exe")

datos = []
try:
    driver = webdriver.Edge(service=service, options=options)
    url = "https://www.entel.pe/hogar/internet/planes"
    driver.get(url)
    time.sleep(8)

    # Buscar todos los contenedores de planes (texto visible)
    cards = driver.find_elements(By.CSS_SELECTOR, "andino-card-planes-hibrida")
    for card in cards:
        try:
            texto = card.text.strip()
            if texto:
                datos.append({"Contenido del plan": texto})
        except Exception as e:
            print(f"⚠️ Error leyendo un plan: {e}")
except Exception as e:
    print(f"❌ Error durante el scraping de Entel: {e}")
finally:
    try:
        driver.quit()
    except Exception:
        pass

# Guardar resultados
df_entel = pd.DataFrame(datos).drop_duplicates()
print(df_entel.head())


                                  Contenido del plan
0  FIBRA ÓPTICA 100 Mbps\n30% dcto.\nOferta Onlin...
1  Plan Destacado\nFIBRA ÓPTICA 300 Mbps\n30% dct...


## Filtrado ENTEL

In [62]:
# Filtrado para planes ENTEL: solo columnas requeridas
import re
import pandas as pd

def extraer_datos_entel(texto):
    texto = texto.lower()

    # Tipo de plan: nunca puede ser igual a la tecnología
    tipo_plan = "No indica"
    for palabra in ["gamer", "plus", "básico", "full", "promo", "promoción"]:
        if palabra in texto and not any(x in texto for x in ["fibra", "hfc", "adsl", "cobre"]):
            tipo_plan = palabra.capitalize()
            break

    # Si el tipo de plan es una tecnología, forzar 'No indica'
    if tipo_plan.lower() in ["fibra óptica", "fibra optica", "hfc", "adsl", "cobre"]:
        tipo_plan = "No indica"

    # Precio regular: buscar explícito o el mayor valor S/ en el texto
    precio_regular = "No indica"
    match_precio = re.search(r"precio regular\s*s?/\s*([\d.,]+)", texto)
    if match_precio:
        precio_regular = f"S/ {match_precio.group(1)}"
    else:
        precios = re.findall(r"s?/\s*([\d.,]+)", texto)
        precios_float = [float(p.replace(',', '.')) for p in precios if p]
        if precios_float:
            precio_regular = f"S/ {max(precios_float):.2f}"

    # Velocidad regular: buscar la mayor velocidad en Mbps
    velocidades = re.findall(r"(\d{2,4})\s*mbps", texto)
    velocidades_int = [int(v) for v in velocidades]
    velocidad_regular = "No indica"
    if velocidades_int:
        velocidad_regular = f"{max(velocidades_int)} Mbps"

    # Tecnología
    tecnologia = "No indica"
    if "fibra" in texto:
        tecnologia = "Fibra óptica"
    elif "adsl" in texto or "cobre" in texto:
        tecnologia = "ADSL/Cobre"
    elif "hfc" in texto:
        tecnologia = "HFC"

    # Extras
    extras = []
    if "mes gratis" in texto:
        extras.append("Mes gratis")
    if "instalación gratis" in texto:
        extras.append("Instalación gratis")
    if "canales" in texto:
        match_canales = re.search(r"(\d+)\s*canales", texto)
        if match_canales:
            extras.append(f"Incluye {match_canales.group(1)} canales")
    if "pago puntual" in texto:
        extras.append("Precio con pago puntual")
    if "dto" in texto or "descuento" in texto:
        extras.append("Descuento especial")
    if "x 6 meses" in texto:
        extras.append("Promoción x 6 meses")
    if "x 3 meses" in texto:
        extras.append("Promoción x 3 meses")
    if "x1 mes" in texto or "x 1 mes" in texto:
        extras.append("Promoción x 1 mes")
    extras = list(dict.fromkeys(extras))
    extras_str = " / ".join(extras) if extras else "Ninguno"

    return {
        "Tipo de plan": tipo_plan,
        "Precio regular": precio_regular,
        "Velocidad regular": velocidad_regular,
        "Tecnología": tecnologia,
        "Extras": extras_str
    }

# Aplicar a tu DataFrame ENTEL (ejemplo)
df_filtrado_entel = df_entel["Contenido del plan"].apply(extraer_datos_entel).apply(pd.Series)
df_filtrado_entel = limpiar_indices(df_filtrado_entel)
print(df_filtrado_entel.head())

  Tipo de plan Precio regular Velocidad regular    Tecnología  \
0    No indica       S/ 70.00          100 Mbps  Fibra óptica   
1    No indica       S/ 90.00          300 Mbps  Fibra óptica   

               Extras  
0  Instalación gratis  
1  Instalación gratis  


# Lectura BITEL

In [49]:
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from selenium.webdriver.edge.options import Options
from selenium.webdriver.common.by import By
import pandas as pd
import time

# Configuración de Edge
options = Options()
# options.add_argument("--headless=new")  # si deseas ejecutar sin ventana
options.add_argument("--disable-gpu")

service = Service(executable_path="C:/WebDrivers/msedgedriver.exe")  # Ajusta si es necesario
driver = webdriver.Edge(service=service, options=options)

# Página de Selectra Bitel
url = "https://ww.selectra.com/pe/telecom/bitel/fibra"
driver.get(url)
time.sleep(10)

# Scroll por seguridad
for _ in range(3):
    driver.execute_script("window.scrollBy(0, 800);")
    time.sleep(1.5)

# Extraer todos los contenedores de planes Bitel
contenedores = driver.find_elements(By.CSS_SELECTOR, "div.tarifa-bitel")

datos = []

for contenedor in contenedores:
    try:
        texto = contenedor.text.strip()
        if texto:
            datos.append({"Contenido del plan": texto})
    except Exception as e:
        print("⚠️ Error leyendo contenedor:", e)

driver.quit()

# Guardar DataFrame
df_bitel = pd.DataFrame(datos).drop_duplicates()
print(df_bitel.head())


                                  Contenido del plan
0  200 Mbps\n100% Fibra Óptica\nS/55.00\nMensual\...
1  400 Mbps\n100% Fibra Óptica\nS/109.00\nMensual...
2  1000 Mbps\n100% Fibra Óptica\nS/139.00\nMensua...


## Filtrado BITEL

In [64]:
# Filtrado mejorado para planes BITEL: columnas requeridas
import re
import pandas as pd

def extraer_datos_bitel(texto):
    texto = texto.lower()

    # Tipo de plan: solo puede ser uno de los siguientes, nunca la tecnología
    tipo_plan = "No indica"
    if "¡más vendido!" in texto or "más vendido" in texto:
        tipo_plan = "Más vendido"
    elif "exclusivo con bitel pack" in texto:
        tipo_plan = "Exclusivo Bitel Pack"
    elif "gamer" in texto:
        tipo_plan = "Gamer"
    elif "plus" in texto:
        tipo_plan = "Plus"
    elif "básico" in texto:
        tipo_plan = "Básico"
    elif "full" in texto:
        tipo_plan = "Full"
    elif "promo" in texto or "promoción" in texto:
        tipo_plan = "Promoción"
    # Si el tipo de plan es una tecnología, forzar 'No indica'
    if tipo_plan.lower() in ["fibra óptica", "fibra optica", "hfc", "adsl", "cobre"]:
        tipo_plan = "No indica"

    # Precio regular: buscar el precio más alto como regular, pero si hay 'paga s/xx.xx mensuales' tomarlo como precio especial
    precio_regular = "No indica"
    precio_especial = None
    match_precio_especial = re.search(r"paga\s*s?/\s*([\d.,]+)\s*mensuales", texto)
    if match_precio_especial:
        precio_especial = f"S/ {match_precio_especial.group(1)}"

    # Buscar todos los precios y tomar el mayor (usualmente el regular)
    precios = re.findall(r"s?/\s*([\d.,]+)", texto)
    precios_float = [float(p.replace(',', '.')) for p in precios if p]
    if precios_float:
        precio_regular = f"S/ {max(precios_float):.2f}"

    # Velocidad regular: buscar la mayor velocidad en Mbps
    velocidades = re.findall(r"(\d{2,4})\s*mbps", texto)
    velocidades_int = [int(v) for v in velocidades]
    velocidad_regular = "No indica"
    if velocidades_int:
        velocidad_regular = f"{max(velocidades_int)} Mbps"

    # Tecnología
    tecnologia = "No indica"
    if "fibra" in texto:
        tecnologia = "Fibra óptica"
    elif "adsl" in texto or "cobre" in texto:
        tecnologia = "ADSL/Cobre"
    elif "hfc" in texto:
        tecnologia = "HFC"

    # Extras
    extras = []
    if "obtén un mes gratis" in texto or "mes gratis" in texto:
        extras.append("Mes gratis")
    if "por el pago adelantado de 3 meses" in texto:
        extras.append("4to mes gratis por pago adelantado")
    if "instalación gratis" in texto:
        extras.append("Instalación gratis")
    if "incluye suscripción streaming" in texto:
        extras.append("Incluye suscripción streaming")
    if "exclusivo con bitel pack" in texto:
        extras.append("Exclusivo con Bitel Pack")
    if precio_especial:
        extras.append(f"Precio especial: {precio_especial}")

    # Unir extras
    extras_str = " / ".join(dict.fromkeys(extras)) if extras else "Ninguno"

    return {
        "Tipo de plan": tipo_plan,
        "Precio regular": precio_regular,
        "Velocidad regular": velocidad_regular,
        "Tecnología": tecnologia,
        "Extras": extras_str
    }

# Aplicar a tu DataFrame BITEL
df_filtrado_bitel = df_bitel["Contenido del plan"].apply(extraer_datos_bitel).apply(pd.Series)
df_filtrado_bitel = limpiar_indices(df_filtrado_bitel)
print(df_filtrado_bitel.head())

           Tipo de plan Precio regular Velocidad regular    Tecnología  \
0             No indica       S/ 55.00          200 Mbps  Fibra óptica   
1  Exclusivo Bitel Pack      S/ 109.00          400 Mbps  Fibra óptica   
2  Exclusivo Bitel Pack      S/ 139.00         1000 Mbps  Fibra óptica   

                                              Extras  
0  Mes gratis / 4to mes gratis por pago adelantad...  
1  Mes gratis / 4to mes gratis por pago adelantad...  
2  Mes gratis / 4to mes gratis por pago adelantad...  


# Unir tablas

In [ ]:
# Unir todos los DataFrames filtrados y agregar columna de proveedor (forzando limpieza de índices antes de unir)
def limpiar_indices(df):
    for col in ['index', 'level_0']:
        if col in df.columns:
            df = df.drop(columns=[col])
    df = df.loc[:,~df.columns.duplicated()]
    return df.reset_index(drop=True)

df_filtrado_claro = limpiar_indices(df_filtrado_claro)
df_filtrado_claro['Proveedor'] = 'Claro'
df_filtrado_movistar = limpiar_indices(df_filtrado_movistar)
df_filtrado_movistar['Proveedor'] = 'Movistar'
df_filtrado_wow = limpiar_indices(df_filtrado_wow)
df_filtrado_wow['Proveedor'] = 'Wow'
df_filtrado_win = limpiar_indices(df_filtrado_win)
df_filtrado_win['Proveedor'] = 'Win'
df_filtrado_entel = limpiar_indices(df_filtrado_entel)
df_filtrado_entel['Proveedor'] = 'Entel'
df_filtrado_bitel = limpiar_indices(df_filtrado_bitel)
df_filtrado_bitel['Proveedor'] = 'Bitel'

# Unir todos los DataFrames en uno solo
df_todas = pd.concat([
    limpiar_indices(df_filtrado_claro),
    limpiar_indices(df_filtrado_movistar),
    limpiar_indices(df_filtrado_wow),
    limpiar_indices(df_filtrado_win),
    limpiar_indices(df_filtrado_entel),
    limpiar_indices(df_filtrado_bitel)
], ignore_index=True)

print(df_todas.head())
df_todas.to_excel('planes_todas_empresas.xlsx', index=False)

   Tipo de plan Precio regular Velocidad regular Tecnología  \
0  Hogar 3 Play      S/ 800.00         1000 Mbps  No indica   
1  Hogar 1 Play      S/ 600.00         1000 Mbps  No indica   
2  Hogar 2 Play      S/ 600.00         1000 Mbps  No indica   
3  Hogar 2 Play      S/ 600.00         1000 Mbps  No indica   
4  Hogar 1 Play      S/ 800.00         1000 Mbps  No indica   

                                              Extras Proveedor  
0  + BONO 1000 MBPS / x 6 meses / para toda la vi...     Claro  
1  + BONO 1000 MBPS / x 6 meses / para toda la vi...     Claro  
2  + BONO 1000 MBPS / x 6 meses / para toda la vi...     Claro  
3  + BONO 1000 MBPS / x 6 meses / para toda la vi...     Claro  
4  + BONO 1000 MBPS / x 6 meses / para toda la vi...     Claro  
